# Stochastic Dominance

To determine if one portfolio stochastically dominates another we first need to compute the CDF.

In [88]:
# defaultdict is a Python dictionary that 
# supports initial values for a key
from collections import defaultdict
import numpy as np

def compute_pdf(time_series):
    d = sorted(time_series)
    di = defaultdict(int)
    inc = 1.0/len(d)
    
    for i in range(len(d)):
        di[d[i]] += 1
    
    val  = []
    prob = []
    
    for k in sorted(di.keys()):
        val.append(k)
        prob.append(inc*di[k])

    return val, prob

In [89]:
# And for the CDF
def compute_cdf(prob):
    cdf = [prob[0]]
    for i in range(1, len(prob)):
        cdf.append(prob[i] + cdf[i-1])
    return cdf


In [90]:
from collections import Counter, OrderedDict

def compute_pdf(data):
    counts = Counter(data)
    n = float(len(data))

    vals = sorted(counts)  # unique values in increasing order
    pdf = OrderedDict((v, counts[v] / n) for v in vals)
    return vals, pdf

def compute_cdf(pdf):
    # pdf is an ordered dict: value -> probability
    cdf = OrderedDict()
    cum = 0.0
    for v, p in pdf.items():
        cum += p
        cdf[v] = cum
    return cdf


data = [1, 2, 4, 6, 8, 9, 1, 4, 5]
val, pdf = compute_pdf(data)
cdf = compute_cdf(pdf)

print(pdf)
print(cdf)


OrderedDict([(1, 0.2222222222222222), (2, 0.1111111111111111), (4, 0.2222222222222222), (5, 0.1111111111111111), (6, 0.1111111111111111), (8, 0.1111111111111111), (9, 0.1111111111111111)])
OrderedDict([(1, 0.2222222222222222), (2, 0.3333333333333333), (4, 0.5555555555555556), (5, 0.6666666666666667), (6, 0.7777777777777779), (8, 0.8888888888888891), (9, 1.0000000000000002)])


In [91]:
from collections import defaultdict
import numpy as np

def compute_pdf(time_series):
    d = sorted(time_series)
    di = defaultdict(int)
    inc = 1.0 / len(d)

    for v in d:
        di[v] += 1

    val, prob = [], []
    for k in sorted(di.keys()):
        val.append(k)
        prob.append(inc * di[k])
    return val, prob

def compute_cdf(prob):
    cdf = [prob[0]]
    for i in range(1, len(prob)):
        cdf.append(prob[i] + cdf[i-1])
    return cdf

In [92]:

def expand_vector(events, x, y):
    # <-- ESTO corrige el KeyError (Series/dict -> lista)
    x = list(x)
    y = list(y)

    index = 0
    d_mod = []
    for pnt in events:
        if index >= len(x):
            d_mod.append(y[-1])
        elif pnt < x[index]:
            d_mod.append(0.0 if index == 0 else y[index-1])
        elif pnt == x[index]:
            d_mod.append(y[index])
        else:
            index += 1
            if index >= len(x):
                d_mod.append(y[-1])
            elif x[index] == pnt:
                d_mod.append(y[index])
            else:
                d_mod.append(y[index-1])
    return d_mod

In [93]:
def check_fosd(d1, d2):
    val1, pdf1 = compute_pdf(d1)
    val2, pdf2 = compute_pdf(d2)
    cdf1, cdf2 = map(compute_cdf, [pdf1, pdf2])

    points = sorted(set(val1 + val2))
    d1_mod = [round(v, 5) for v in expand_vector(points, val1, cdf1)]
    d2_mod = [round(v, 5) for v in expand_vector(points, val2, cdf2)]

    d1_fosd_d2 = all(x <= y for x, y in zip(d1_mod, d2_mod))
    d2_fosd_d1 = all(x >= y for x, y in zip(d1_mod, d2_mod))
    return d1_fosd_d2, d2_fosd_d1

In [94]:
def check_sosd(d1, d2):
    val1, pdf1 = compute_pdf(d1)
    val2, pdf2 = compute_pdf(d2)
    cdf1, cdf2 = map(compute_cdf, [pdf1, pdf2])

    points = sorted(set(val1 + val2))
    d1_mod = [round(v, 5) for v in expand_vector(points, val1, cdf1)]
    d2_mod = [round(v, 5) for v in expand_vector(points, val2, cdf2)]

    d1_areas = np.cumsum([d1_mod[i] * (points[i+1] - points[i]) for i in range(len(points)-1)])
    d2_areas = np.cumsum([d2_mod[i] * (points[i+1] - points[i]) for i in range(len(points)-1)])

    d1_sosd_d2 = all(x <= y for x, y in zip(d1_areas, d2_areas))
    d2_sosd_d1 = all(x >= y for x, y in zip(d1_areas, d2_areas))
    return d1_sosd_d2, d2_sosd_d1


In [95]:
d1 = [80, 80, 30, 30, 30, 60, 50, 50, 50, 50]
d2 = [10, 10, 50, 50, 50, 70, 30, 30, 30, 30]
d3 = [20, 80]
d4 = [0, 100]

print(check_fosd(d1, d2))
print(check_sosd(d1, d2))
print(check_fosd(d3, d4))
print(check_sosd(d3, d4))


(True, False)
(True, False)
(False, False)
(True, False)
